In [77]:
from os import mkdir
import numpy as np
import random
import matplotlib.pyplot as plt
from scipy.io.wavfile import write

In [79]:
#Creating the folder in a specified path

setID= 'a'
mkdir(f"C:/Users/Laptop Elitebook/Documents/NEUROCIENCIAS/practicas-tesis/Audios/set_{setID}")


FileExistsError: [WinError 183] No se puede crear un archivo que ya existe: 'C:/Users/Laptop Elitebook/Documents/NEUROCIENCIAS/practicas-tesis/Audios/set_a'

In [80]:
output_path = f"C:/Users/Laptop Elitebook/Documents/NEUROCIENCIAS/practicas-tesis/Audios/set_{setID}"


In [87]:
#Defining variables

ntokenreps = 10  #how many times the target repeats in a trial
tonedur = 2  #original/correct is 0.05 
griddt = 0.025 #time resolution of our grid
gridfs = 1/griddt
minf = 200
maxf = 12800
noct = 6
spectral_density = [4] #tones per octave   #Should we leave it as a list, make it an array or just like the number?
temporal_density = [4] #tones per second
temp_dens_ptonepergriddt = [temp / gridfs for temp in temporal_density] #probability of a tone in each griddt
tempi = [0.5] #ms, the inter-repeat interval of tokens 
tokendur = 0.2 #rget duration


fs = 44100 #generate sounds with this fs
fs48 = 48828 #convert to this fs for TDT equipment
maxf48k = fs48/2
trialseed = 888
mintokenstart = 1 
maxtokenstart = 1.5 #irst token starts between 1-1.5s into (random) tone cloud
np.random.seed(trialseed)  #command to set the seed
ntrialspercond = 60
stimdb = 85
targetrms = (10**(-94/20))*(10**(stimdb/20)) #convertion of dB to amplitude. the number we would use
targetrms_wav = 0.1 #0.4

In [88]:
#compute hann envelope (for creating a smooth on and off ramp to sounds)

hanndur = 0.05 #s #original/correct is 0.005
thann = np.arange(1/fs, hanndur+1/fs, 1/fs)
ramp = np.cos(2*np.pi*0.5/hanndur*thann)
ttone = np.arange(1/fs, tonedur+1/fs, 1/fs)
env = np.ones(ttone.shape)
env[:len(ramp)] = np.flip(ramp)
env[-len(ramp):] = ramp
env=(env+1)/2
#plt.plot(ttone, env)

In [89]:
#now loop through conditions and trials to generate
trialct = 0 
sp_dens = [] 
tp_dens = []
trialnr = []
tempo = []
play_grid = []
latency_grid = []

#to save params for manipulations/variations
g_play = []
g_latency = []
g_freq = []
t_play = []
t_latency = []
t_freq = []
t_startid = []
t_ioi = []
g_freqlist = []
g_seqdur = []
g_setIDs = []
g_specdens = []
g_tempdens = []

In [90]:
for s in range(len(spectral_density)):
    specdens = spectral_density[s]
    freq_list = np.logspace(np.log10(minf),np.log10(maxf),specdens*noct+1)
    for t in range(len(temporal_density)):
        tempdens = temp_dens_ptonepergriddt[s]
        for n in range(ntrialspercond):
            for N in range(len(tempi)):
                trialct = trialct+1
                startt = round((maxtokenstart-mintokenstart)/griddt*np.random.rand()) + mintokenstart/griddt # time of first token start in units of n griddt
                #generating sound
                ftone = np.random.uniform(minf, maxf) #pick a random tone frequency between minf and maxf
                amp = np.sin(2*np.pi*ftone*ttone) #make the tone
                amp = amp*env #hanning env already calculated outside
                wav_filename = f"{output_path}/trial_{trialct}_freq_{int(ftone)}_Hz.wav"
                write(wav_filename, fs, amp.astype(np.float32))
                txt_filename = f"{output_path}/trial_{trialct}_freq_{int(ftone)}.txt"
                np.savetxt(txt_filename, amp, fmt='%0.6f')
                